OK so in this notebook I was about to tackle some tougher dataset. 

To be honest,  just plain KNN did a pretty good job of clustering the data right off the bat, it really wasn't all that difficult. 
So I picked another dataset from the UC Irvine data repository that was an employee churn dataset.

Since there were many categorical variables, I decided to format it so that I could feed it into a mathematical function.
First order of business was to load the libraries:

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#Nescessary to display all columns for some reason
from IPython.core.display import HTML


https://towardsdatascience.com/categorical-encoding-using-label-encoding-and-one-hot-encoder-911ef77fb5bd

https://towardsdatascience.com/guide-to-encoding-categorical-features-using-scikit-learn-for-machine-learning-5048997a5c79

Then I loaded the data:

In [3]:
pd.set_option('display.max_colwidth', None)

csv = pd.read_csv("e_attr1.csv")
data = csv.copy()

To get an idea of what the data contained, I decided to take a look at it:

In [4]:
display(HTML(csv.head().to_html()))

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,41,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Yes,11,3,1,80,0,8,0,1,6,4,0,5,Yes
1,49,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,No,23,4,4,80,1,10,3,3,10,7,1,7,No
2,37,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Yes,15,3,2,80,0,7,3,3,0,0,0,0,Yes
3,33,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,4,Female,56,3,1,Research Scientist,3,Married,2909,23159,1,Yes,11,3,3,80,0,8,3,3,8,7,3,0,No
4,27,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,16632,9,No,12,3,4,80,1,6,3,3,2,2,2,2,No


In [6]:
categories=['BusinessTravel','MaritalStatus','JobRole','Gender','EducationField','Department','OverTime']

for i in categories:
    print(i)
    print(csv[i].value_counts())
    print("-----------------")

BusinessTravel
Travel_Rarely        1043
Travel_Frequently     277
Non-Travel            150
Name: BusinessTravel, dtype: int64
-----------------
MaritalStatus
Married     673
Single      470
Divorced    327
Name: MaritalStatus, dtype: int64
-----------------
JobRole
Sales Executive              326
Research Scientist           292
Laboratory Technician        259
Manufacturing Director       145
Healthcare Representative    131
Manager                      102
Sales Representative          83
Research Director             80
Human Resources               52
Name: JobRole, dtype: int64
-----------------
Gender
Male      882
Female    588
Name: Gender, dtype: int64
-----------------
EducationField
Life Sciences       606
Medical             464
Marketing           159
Technical Degree    132
Other                82
Human Resources      27
Name: EducationField, dtype: int64
-----------------
Department
Research & Development    961
Sales                     446
Human Resources           

I decided not to split the "Business travel" category as there was an obvious numerical mapping. Instead, I just mapped the to sensible integers:

In [7]:
data['BusinessTravel'] = data['BusinessTravel'].map({'Non-Travel':0,
                           'Travel_Rarely':1,
                           'Travel_Frequently':2}
                          )

One hot encoding is splitting each type of a category . 

The rest I decided to one-hot encode else the prior scheme would have implied a hierarchy that wasn't there.
(a man is not superior to a woman or vice versa).
The dataset also wasn't very big and was unlikely to cause memory issues.

In [8]:
def OneHot(columns_names,data):
    for name in columns_names:
        ohe = OneHotEncoder(sparse = False)
        encoded_df =  pd.DataFrame(ohe.fit_transform(data[[name]])).copy()
        encoded_df.columns = ohe.categories_
        
        #axis =1 ----> refers to column 
        data = data.drop([name],axis=1)
        data = pd.concat([encoded_df,data],axis=1)

    return data


column_names = ["Department","EducationField","JobRole", "Gender","MaritalStatus"]
        
data = OneHot(column_names,data)

display(HTML(data.head().to_html()))    
    
    

,"(Divorced,)","(Married,)","(Single,)","(Female,)","(Male,)","(Healthcare Representative,)","(Human Resources,)","(Laboratory Technician,)","(Manager,)","(Manufacturing Director,)","(Research Director,)","(Research Scientist,)","(Sales Executive,)","(Sales Representative,)","(Human Resources,)","(Life Sciences,)","(Marketing,)","(Medical,)","(Other,)","(Technical Degree,)","(Human Resources,)","(Research & Development,)","(Sales,)",Age,BusinessTravel,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,MonthlyRate,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,41,1,1102,1,2,1,1,2,94,3,2,4,5993,19479,8,Yes,11,3,1,80,0,8,0,1,6,4,0,5,Yes
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,49,2,279,8,1,1,2,3,61,2,2,2,5130,24907,1,No,23,4,4,80,1,10,3,3,10,7,1,7,No
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,37,1,1373,2,2,1,4,4,92,2,1,3,2090,2396,6,Yes,15,3,2,80,0,7,3,3,0,0,0,0,Yes
3,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,33,2,1392,3,4,1,5,4,56,3,1,3,2909,23159,1,Yes,11,3,3,80,0,8,3,3,8,7,3,0,No
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,27,1,591,2,1,1,7,1,40,3,1,2,3468,16632,9,No,12,3,4,80,1,6,3,3,2,2,2,2,No


I can do the same as I did above with the 'BusinessTravel' category. 

In [9]:
def Label(column_names,data):
    for name in column_names:
        lab = LabelEncoder() #https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
        encoded_df =  pd.DataFrame(lab.fit_transform(data[[name]])).copy()
        encoded_df.columns = [name]
        data[name] = encoded_df
        
        #axis =1 ----> refers to column 
    return data

column_names = ["OverTime"]
data = Label(column_names,data)

display(HTML(data.head().to_html()))    



/home/flackjacket/miniconda3/envs/machine-learning-env/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,"(Divorced,)","(Married,)","(Single,)","(Female,)","(Male,)","(Healthcare Representative,)","(Human Resources,)","(Laboratory Technician,)","(Manager,)","(Manufacturing Director,)","(Research Director,)","(Research Scientist,)","(Sales Executive,)","(Sales Representative,)","(Human Resources,)","(Life Sciences,)","(Marketing,)","(Medical,)","(Other,)","(Technical Degree,)","(Human Resources,)","(Research & Development,)","(Sales,)",Age,BusinessTravel,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,MonthlyRate,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,41,1,1102,1,2,1,1,2,94,3,2,4,5993,19479,8,1,11,3,1,80,0,8,0,1,6,4,0,5,Yes
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,49,2,279,8,1,1,2,3,61,2,2,2,5130,24907,1,0,23,4,4,80,1,10,3,3,10,7,1,7,No
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,37,1,1373,2,2,1,4,4,92,2,1,3,2090,2396,6,1,15,3,2,80,0,7,3,3,0,0,0,0,Yes
3,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,33,2,1392,3,4,1,5,4,56,3,1,3,2909,23159,1,1,11,3,3,80,0,8,3,3,8,7,3,0,No
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,27,1,591,2,1,1,7,1,40,3,1,2,3468,16632,9,0,12,3,4,80,1,6,3,3,2,2,2,2,No


In [10]:
lab = LabelEncoder()
encoded_last_column =  pd.DataFrame(lab.fit_transform(data[['Attrition']])).copy()
encoded_last_column.columns = ['Attrition']

data['Attrition'] = encoded_last_column


/home/flackjacket/miniconda3/envs/machine-learning-env/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
display(HTML(data.head().to_html()))    


,"(Divorced,)","(Married,)","(Single,)","(Female,)","(Male,)","(Healthcare Representative,)","(Human Resources,)","(Laboratory Technician,)","(Manager,)","(Manufacturing Director,)","(Research Director,)","(Research Scientist,)","(Sales Executive,)","(Sales Representative,)","(Human Resources,)","(Life Sciences,)","(Marketing,)","(Medical,)","(Other,)","(Technical Degree,)","(Human Resources,)","(Research & Development,)","(Sales,)",Age,BusinessTravel,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,MonthlyRate,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,41,1,1102,1,2,1,1,2,94,3,2,4,5993,19479,8,1,11,3,1,80,0,8,0,1,6,4,0,5,1
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,49,2,279,8,1,1,2,3,61,2,2,2,5130,24907,1,0,23,4,4,80,1,10,3,3,10,7,1,7,0
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,37,1,1373,2,2,1,4,4,92,2,1,3,2090,2396,6,1,15,3,2,80,0,7,3,3,0,0,0,0,1
3,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,33,2,1392,3,4,1,5,4,56,3,1,3,2909,23159,1,1,11,3,3,80,0,8,3,3,8,7,3,0,0
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,27,1,591,2,1,1,7,1,40,3,1,2,3468,16632,9,0,12,3,4,80,1,6,3,3,2,2,2,2,0


In [12]:
data.to_csv('encoded_data.csv',index=False)